##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Guia completo de treinamento consciente de quantização

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/model_optimization/guide/quantization/training_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/model_optimization/guide/quantization/training_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/model_optimization/guide/quantization/training_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Bem-vindo ao guia completo sobre treinamento consciente de quantização do Keras.

Esta página documenta vários casos de uso e mostra como usar a API para cada um. Depois de saber quais APIs são necessárias, encontre os parâmetros e os detalhes de nível inferior na [documentação da API](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/quantization).

- Se quiser ver os benefícios do treinamento consciente de quantização e saber o que é compatível, confira a [visão geral](https://www.tensorflow.org/model_optimization/guide/quantization/training.md).
- Para um único exemplo completo, veja o [exemplo de treinamento consciente de quantização](https://www.tensorflow.org/model_optimization/guide/quantization/training_example.md).

São abordados os seguintes casos de uso:

- Implante um modelo com a quantização de 9 bits seguindo estas etapas.
    - Defina um modelo consciente de quantização.
    - Somente para modelos HDF5 do Keras, use lógica especial de checkpoint e desserialização. O resto do treinamento é padrão.
    - Crie um modelo quantizado a partir do consciente de quantização.
- Experimente com a quantização.
    - Qualquer experimentação não tem caminho compatível para implantação.
    - As camadas personalizadas do Keras se enquadram na experimentação.

## Configuração

Para encontrar as APIs de que você precisa e para fins de compreensão, você pode executar mas pular a leitura desta seção.

In [ ]:
! pip install -q tensorflow
! pip install -q tensorflow-model-optimization

import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot

import tempfile

input_shape = [20]
x_train = np.random.randn(1, 20).astype(np.float32)
y_train = tf.keras.utils.to_categorical(np.random.randn(1), num_classes=20)

def setup_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(20, input_shape=input_shape),
      tf.keras.layers.Flatten()
  ])
  return model

def setup_pretrained_weights():
  model= setup_model()

  model.compile(
      loss=tf.keras.losses.categorical_crossentropy,
      optimizer='adam',
      metrics=['accuracy']
  )

  model.fit(x_train, y_train)

  _, pretrained_weights = tempfile.mkstemp('.tf')

  model.save_weights(pretrained_weights)

  return pretrained_weights

def setup_pretrained_model():
  model = setup_model()
  pretrained_weights = setup_pretrained_weights()
  model.load_weights(pretrained_weights)
  return model

setup_model()
pretrained_weights = setup_pretrained_weights()

Defina um modelo consciente de quantização

Ao definir modelos das maneiras a seguir, há caminhos disponíveis para a implementação nos back-ends listados na [página de visão geral](https://www.tensorflow.org/model_optimization/guide/quantization/training.md). Por padrão, é usada a quantização de 8 bits.

Observação: na verdade, um modelo consciente de quantização não é quantizado. A criação do modelo quantizado é um passo separado.

### Quantize o modelo inteiro

**Seu caso de uso:**

- Os modelos com subclasses não são compatíveis.

**Dicas para melhor exatidão do modelo:**

- Tente "quantizar algumas camadas" para pular a quantização das camadas que mais reduzem a exatidão.
- Geralmente, é melhor fazer ajustes com o treinamento consciente de quantização, em vez de treinar do zero.


Para tornar o modelo inteiro consciente de quantização, aplique `tfmot.quantization.keras.quantize_model` a ele.


In [ ]:
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended for model accuracy

quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)
quant_aware_model.summary()

### Quantize algumas camadas

A quantização de um modelo pode ter um efeito negativo na exatidão. Você pode quantizar camadas selecionadas de um modelo para explorar o trade-off entre exatidão, velocidade e tamanho do modelo.

**Seu caso de uso:**

- Para implantar em um back-end que só funciona bem com modelos totalmente quantizados (por exemplo, EdgeTPU v1 e a maioria dos DSPs), tente "Quantizar o modelo inteiro".

**Dicas para melhor exatidão do modelo:**

- Geralmente, é melhor fazer ajustes com o treinamento consciente de quantização, em vez de treinar do zero.
- Tente quantizar as camadas finais, em vez das iniciais.
- Evite quantizar camadas críticas (por exemplo, mecanismo de atenção).


No exemplo abaixo, quantize apenas as camadas `Dense`.

In [ ]:
# Create a base model
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended for model accuracy

# Helper function uses `quantize_annotate_layer` to annotate that only the 
# Dense layers should be quantized.
def apply_quantization_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_quantization_to_dense` 
# to the layers of the model.
annotated_model = tf.keras.models.clone_model(
    base_model,
    clone_function=apply_quantization_to_dense,
)

# Now that the Dense layers are annotated,
# `quantize_apply` actually makes the model quantization aware.
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
quant_aware_model.summary()

Enquanto esse exemplo usa o tipo de camada para decidir o que quantizar, a maneira mais fácil de quantizar uma camada específica é definir sua propriedade `name` e procurar esse nome na `clone_function`.

In [ ]:
print(base_model.layers[0].name)

#### Exatidão do modelo mais legível, mas possivelmente mais baixa

Isto não é compatível com o treinamento consciente de quantização. Por isso, pode apresentar menos exatidão do que os exemplos acima.

**Exemplo funcional**

In [ ]:
# Use `quantize_annotate_layer` to annotate that the `Dense` layer
# should be quantized.
i = tf.keras.Input(shape=(20,))
x = tfmot.quantization.keras.quantize_annotate_layer(tf.keras.layers.Dense(10))(i)
o = tf.keras.layers.Flatten()(x)
annotated_model = tf.keras.Model(inputs=i, outputs=o)

# Use `quantize_apply` to actually make the model quantization aware.
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)

# For deployment purposes, the tool adds `QuantizeLayer` after `InputLayer` so that the
# quantized model can take in float inputs instead of only uint8.
quant_aware_model.summary()

**Exemplo sequencial**


In [ ]:
# Use `quantize_annotate_layer` to annotate that the `Dense` layer
# should be quantized.
annotated_model = tf.keras.Sequential([
  tfmot.quantization.keras.quantize_annotate_layer(tf.keras.layers.Dense(20, input_shape=input_shape)),
  tf.keras.layers.Flatten()
])

# Use `quantize_apply` to actually make the model quantization aware.
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)

quant_aware_model.summary()

## Faça o checkpoint e desserialize

**Seu caso de uso:** este código só é necessário para o formato de modelo HDF5 (e não pesos HDF5 ou outros formatos).

In [ ]:
# Define the model.
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended for model accuracy
quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)

# Save or checkpoint the model.
_, keras_model_file = tempfile.mkstemp('.h5')
quant_aware_model.save(keras_model_file)

# `quantize_scope` is needed for deserializing HDF5 models.
with tfmot.quantization.keras.quantize_scope():
  loaded_model = tf.keras.models.load_model(keras_model_file)

loaded_model.summary()

## Crie e implante o modelo quantizado

Em geral, consulte a documentação para o back-end de implementação que você usará.

Este é um exemplo do back-end do TFLite.

In [ ]:
base_model = setup_pretrained_model()
quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)

# Typically you train the model here.

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

## Experimente com a quantização

**Seu caso de uso**: com o uso das APIs a seguir, não há caminho compatível para a implantação. Por exemplo, a conversão para o TFLite e as implementações do kernel só são compatíveis com a quantização de 8 bits. Os recursos também são experimentais e não estão sujeitos à compatibilidade com versões anteriores.

- `tfmot.quantization.keras.QuantizeConfig`
- `tfmot.quantization.keras.quantizers.Quantizer`
- `tfmot.quantization.keras.quantizers.LastValueQuantizer`
- `tfmot.quantization.keras.quantizers.MovingAverageQuantizer`

### Configuração: DefaultDenseQuantizeConfig

A experimentação exige o uso de `tfmot.quantization.keras.QuantizeConfig`, que descreve como quantizar os pesos, as ativações e as saídas de uma camada.

O exemplo abaixo define a mesma `QuantizeConfig` usada para a camada `Dense` nos padrões da API.

Durante a propagação para a frente nesse exemplo, o `LastValueQuantizer` retornado em `get_weights_and_quantizers` é chamado com `layer.kernel` como a entrada, gerando uma saída. A saída substitui o `layer.kernel` na propagação para a frente original da camada `Dense`, pela lógica definida em `set_quantize_weights`. A mesma ideia se aplica às ativações e saídas.


In [ ]:
LastValueQuantizer = tfmot.quantization.keras.quantizers.LastValueQuantizer
MovingAverageQuantizer = tfmot.quantization.keras.quantizers.MovingAverageQuantizer

class DefaultDenseQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    # Configure how to quantize weights.
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

    # Configure how to quantize activations.
    def get_activations_and_quantizers(self, layer):
      return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
      layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
      layer.activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
      return []

    def get_config(self):
      return {}

### Quantize camadas personalizadas do Keras


Esse exemplo usa a `DefaultDenseQuantizeConfig` para quantizar a `CustomLayer`.

A aplicação da configuração é a mesma em todos os casos de uso da seção "Experimente com a quantização".

- Aplique `tfmot.quantization.keras.quantize_annotate_layer` a `CustomLayer` e passe a `QuantizeConfig`.
- Use `tfmot.quantization.keras.quantize_annotate_model` para continuar a quantizar o resto do modelo com os padrões da API.


In [ ]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class CustomLayer(tf.keras.layers.Dense):
  pass

model = quantize_annotate_model(tf.keras.Sequential([
   quantize_annotate_layer(CustomLayer(20, input_shape=(20,)), DefaultDenseQuantizeConfig()),
   tf.keras.layers.Flatten()
]))

# `quantize_apply` requires mentioning `DefaultDenseQuantizeConfig` with `quantize_scope`
# as well as the custom Keras layer.
with quantize_scope(
  {'DefaultDenseQuantizeConfig': DefaultDenseQuantizeConfig,
   'CustomLayer': CustomLayer}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

### Modifique os parâmetros de quantização


**Erro comum:** quantizar o bias para menos que 32 bits geralmente prejudica muito a exatidão do modelo.

Este exemplo modifica a camada `Dense` para que use os 4 bits nos seus pesos em vez dos 8 bits padrão. O resto do modelo continua usando os padrões da API.


In [ ]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class ModifiedDenseQuantizeConfig(DefaultDenseQuantizeConfig):
    # Configure weights to quantize with 4-bit instead of 8-bits.
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, LastValueQuantizer(num_bits=4, symmetric=True, narrow_range=False, per_axis=False))]

A aplicação da configuração é a mesma em todos os casos de uso da seção "Experimente com a quantização".

- Aplique `tfmot.quantization.keras.quantize_annotate_layer` à camada `Dense` e passe a `QuantizeConfig`.
- Use `tfmot.quantization.keras.quantize_annotate_model` para continuar a quantizar o resto do modelo com os padrões da API.

In [ ]:
model = quantize_annotate_model(tf.keras.Sequential([
   # Pass in modified `QuantizeConfig` to modify this Dense layer.
   quantize_annotate_layer(tf.keras.layers.Dense(20, input_shape=(20,)), ModifiedDenseQuantizeConfig()),
   tf.keras.layers.Flatten()
]))

# `quantize_apply` requires mentioning `ModifiedDenseQuantizeConfig` with `quantize_scope`:
with quantize_scope(
  {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

### Modifique partes da camada para quantizar


Este exemplo modifica a camada `Dense` para pular a quantização da ativação. O resto do modelo continua usando os padrões da API.

In [ ]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class ModifiedDenseQuantizeConfig(DefaultDenseQuantizeConfig):
    def get_activations_and_quantizers(self, layer):
      # Skip quantizing activations.
      return []

    def set_quantize_activations(self, layer, quantize_activations):
      # Empty since `get_activaations_and_quantizers` returns
      # an empty list.
      return

A aplicação da configuração é a mesma em todos os casos de uso da seção "Experimente com a quantização".

- Aplique `tfmot.quantization.keras.quantize_annotate_layer` à camada `Dense` e passe a `QuantizeConfig`.
- Use `tfmot.quantization.keras.quantize_annotate_model` para continuar a quantizar o resto do modelo com os padrões da API.

In [ ]:
model = quantize_annotate_model(tf.keras.Sequential([
   # Pass in modified `QuantizeConfig` to modify this Dense layer.
   quantize_annotate_layer(tf.keras.layers.Dense(20, input_shape=(20,)), ModifiedDenseQuantizeConfig()),
   tf.keras.layers.Flatten()
]))

# `quantize_apply` requires mentioning `ModifiedDenseQuantizeConfig` with `quantize_scope`:
with quantize_scope(
  {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

### Use o algoritmo de quantização personalizado


A classe `tfmot.quantization.keras.quantizers.Quantizer` é uma invocável que pode aplicar qualquer algoritmo às suas entradas.

Neste exemplo, as entradas são os pesos, e aplicamos a matemática na função __call__ `FixedRangeQuantizer` aos pesos. Em vez dos valores originais dos pesos, a saída de `FixedRangeQuantizer` é passada ao que usaria os pesos.

In [ ]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class FixedRangeQuantizer(tfmot.quantization.keras.quantizers.Quantizer):
  """Quantizer which forces outputs to be between -1 and 1."""

  def build(self, tensor_shape, name, layer):
    # Not needed. No new TensorFlow variables needed.
    return {}

  def __call__(self, inputs, training, weights, **kwargs):
    return tf.keras.backend.clip(inputs, -1.0, 1.0)

  def get_config(self):
    # Not needed. No __init__ parameters to serialize.
    return {}


class ModifiedDenseQuantizeConfig(DefaultDenseQuantizeConfig):
    # Configure weights to quantize with 4-bit instead of 8-bits.
    def get_weights_and_quantizers(self, layer):
      # Use custom algorithm defined in `FixedRangeQuantizer` instead of default Quantizer.
      return [(layer.kernel, FixedRangeQuantizer())]

A aplicação da configuração é a mesma em todos os casos de uso da seção "Experimente com a quantização".

- Aplique `tfmot.quantization.keras.quantize_annotate_layer` à camada `Dense` e passe a `QuantizeConfig`.
- Use `tfmot.quantization.keras.quantize_annotate_model` para continuar a quantizar o resto do modelo com os padrões da API.

In [ ]:
model = quantize_annotate_model(tf.keras.Sequential([
   # Pass in modified `QuantizeConfig` to modify this `Dense` layer.
   quantize_annotate_layer(tf.keras.layers.Dense(20, input_shape=(20,)), ModifiedDenseQuantizeConfig()),
   tf.keras.layers.Flatten()
]))

# `quantize_apply` requires mentioning `ModifiedDenseQuantizeConfig` with `quantize_scope`:
with quantize_scope(
  {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()